# Project Case:
### Olist Store is the largest department store in Brazilian marketplace
Olist connects small businesses from all over Brazil to channels without hassle and with a single contract The Brazilian
ecommerce public dataset of orders (from 2016 to 2018 made at Olist Store is provided to your
company for analysis

Your manager is asking you to critically analyse the provided datasets using Business Intelligence
tools and provide some marketing findings recommendations in a report format The dataset has
information of 100 k orders made at multiple marketplaces in Brazil Its features allow viewing an
order from multiple dimensions from order status, price, payment and freight performance to
customer location, product attributes and finally reviews written by customers A geolocation
dataset that relates Brazilian zip codes to lat lng coordinates is also integrated in the dataset

After a customer purchases the product from Olist Store, a seller gets notified to fulfill that order
Once the customer receives the product, or the estimated delivery date is due, the customer gets
a satisfaction survey by email where they can give a note for the purchase experience and write
down some comments


## Background:
###  CEP: the Brazilian Zip Code
A brazilian zip code, also know as CEP, stands for Postal Adressing Code (Código de Endereçamento Postal) and contains 8 digits. Introduced in 1972 as a sequence of five digits, it was expanded to eight digits in 1992 to allow for more precise localization. The standard format is "nnnnn-nnn" (the original five digits, an hyphen, and the new three digits).

**CEP**: 12.345-678

Most cities with population around 100,000 and above have a CEP assigned to every public place and to some high-occupancy private spaces, like major commercial buildings and large residential condos. Small towns are assigned a general 5-digit code followed by the suffix -000.

1. the first part is composed by 5 digits that represent Region, Subregion, Sector, Subsector and Subsector Splitter.
2. the second part contain 3 digits, separated by an hyphen from the first, and it represents the Distribution Identifiers.

More info here: https://www.correios.com.br/a-a-z/cep-codigo-de-enderecamento-postal

### Geolocation dataset:
The first 5 digits of the zip code are provided, together with latitude and longitude information, city name and state name.

#### Issues:
- Since we only have the first 5 digits of the zip code, named zip_code_prefix, precise location cannot be determined. This also explains why there are varying lat/lng information for each zip_code_prefix.
- Portugese characters include characters which are represented differently from english characters. City names are not uniformly respresented in a single way in the dataset. This is the main challenge to clean up and transform.

#### Approach:
- Find a reliable source of City, State Names together with CEP information where available, and use this to clean/transform the names in the dataset


# Final Round of Transformation
String matching with a list of city names from a dataset retrieved from here:
https://www.kaggle.com/datasets/crisparada/brazilian-cities?resource=download

And finally we load the data into DB


In [ ]:
import os
import pandas as pd
import numpy as np
import time
import sqlalchemy as db
from dotenv import load_dotenv
from thefuzz import process, fuzz
from datetime import date

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Access environment variable
db_local=os.environ['olist_db_local']
db_azure=os.environ['olist_db_azure']
back4app_id=os.environ['back4app_id']
back4app_key=os.environ['back4app_key']
export_path_local=os.environ['export_path_local']

# Import and Inspect Processed Location Datasets

In [ ]:
# Download processed location data, convert to a dataframe
geo_df = pd.read_csv(f'{export_path_local}geolocation_2024-04-28.csv')
geo_df.info()

In [ ]:
# Look at first 5 entries
geo_df.head()

In [ ]:
# Zip code prefix should be 5 digits long with leading zeroes
geo_df['zip_code_prefix'] = geo_df['zip_code_prefix'].apply(lambda x: '{0:0>5}'.format(x))

In [ ]:
# Drop old index col
geo_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# Drop cep col - not accurate after merge due to incomplete original zip code data
geo_df.drop('cep', axis=1, inplace=True)
geo_df.info()

In [ ]:
# Inspect unique values in city (1)
city = geo_df['city'].unique().tolist()
print('Unique city values: ', geo_df['city'].nunique(), '\n', city)

# Import & Inspect City Dataset

In [ ]:
# Import list of Brazillian cities
cities_df = pd.read_csv(f'{export_path_local}BRAZIL_CITIES_REV2022.CSV')
cities_df.info()

In [ ]:
# Create a list of city names
cities = cities_df['CITY'].tolist()
print(len(cities), '\n', cities)

In [ ]:
# Test for names in list
if 'São Paulo' in cities:
    print("Desired item is in list")

In [ ]:
# Test matching ratio
fuzz.WRatio('São Paulo', 'Sao Paulo')

In [ ]:
# Inspect match statement output
matches = process.extract('Abadia Dos Dourados', geo_df['city'], limit = geo_df.shape[0])
matches

# Using City dataset, perform fuzzy string-match with processed location dataset

In [ ]:
%%time
for i, city in enumerate(cities):
    matches = process.extract(city, geo_df['city'], limit=geo_df.shape[0])
    high_score_match = None

    # Iterate through the matches to find the one with the highest score above 90
    for potential_match in matches:
        if potential_match[1] > 91:
            if high_score_match is None or potential_match[1] > high_score_match[1]:
                high_score_match = potential_match

    # If a match with the highest score above 80 is found, update the 'geolocation_city'
    if high_score_match:
        print(high_score_match[0])
        geo_df.loc[geo_df['city'] == high_score_match[0], 'city'] = city
    n = geo_df[geo_df['city'] == city]['city'].count()
    print(f'{i}', '. ', city, '\t','Matched: ', n)

In [ ]:
# Inspect unique 'city' values in processed df
city = geo_df['city'].unique().tolist()
print('Unique city values: ', geo_df['city'].nunique(), '\n', city)

In [ ]:
# Replace 'Sao' with 'São' in the 'column_name'
geo_df['city'] = geo_df['city'].str.replace('Sao', 'São')

In [ ]:
# Inspect list of cities
cities = geo_df['city'].unique().tolist()
cities.sort()
print('Unique city values: ', geo_df['city'].nunique(), '\n', cities)

In [ ]:
# Perform a similarity check on adjacent elements
city_1 = []
city_2 = []

for i in range(len(cities)-1):
    match = fuzz.WRatio(cities[i], cities[i+1])
    if match > 92:
        city_1.append(cities[i])
        city_2.append(cities[i+1])

print(city_1, city_2)

In [ ]:
for i in range(len(city_1)):
    print(i,' ',city_1[i],'\n',city_2[i])

In [ ]:
# Remove different cities from the lists
index = [1, 4, 9, 38, 54, 60, 61, 64, 65, 68]
for i in sorted(index, reverse=True):
    city_1.pop(i)
    city_2.pop(i)

In [ ]:
# Replace similar city names with the same name 
geo_df['city'].replace(to_replace=city_1, value=city_2, inplace=True)

# FROM HERE

In [ ]:
# # Download processed location data, convert to a dataframe
# geo_df = pd.read_csv(f'{export_path_local}geolocation_realFuzzy_2024-04-30.csv')
# # Zip code prefix should be 5 digits long with leading zeroes
# geo_df['zip_code_prefix'] = geo_df['zip_code_prefix'].apply(lambda x: '{0:0>5}'.format(x))
# # Drop old index col
# geo_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
geo_df.describe()

In [ ]:
geo_df.info()

# Remove outliers in geolocation data:
- most Northern spot is at 5 deg 16′ 27.8″ N latitude
- most southern spot is at 33 deg, 45′ 04.21″ S Latitude.
- most Eastern spot is 34 deg, 47′ 35.33″ W Long
- most Western spot is at 73 deg, 58′ 58.19″W Long
- https://en.wikipedia.org/wiki/Geography_of_Brazil

In [ ]:
# Removing some outliers
#Brazils most Northern spot is at 5 deg 16′ 27.8″ N latitude.;
geo_df = geo_df[geo_df['lat'] <= 5.27438888]
#it’s most Western spot is at 73 deg, 58′ 58.19″W Long.
geo_df = geo_df[geo_df['lng'] >= -73.98283055]
#It’s most southern spot is at 33 deg, 45′ 04.21″ S Latitude.
geo_df = geo_df[geo_df['lat'] >= -33.75116944]
#It’s most Eastern spot is 34 deg, 47′ 35.33″ W Long.
geo_df = geo_df[geo_df['lng'] <=  -34.79314722]

In [ ]:
geo_df.info()

In [ ]:
geo_df.describe()

In [ ]:
geo_df.nunique()

In [ ]:
geo_df['state'] = geo_df['state'].str.strip()
geo_df['state'].nunique()

In [ ]:
geo_df.head()

In [ ]:
# Export to .csv file
today = date.today()
geo_df.to_csv(f'{export_path_local}geolocation_realFuzzy_{today}.csv')

# Create table in db

In [ ]:
# Connect to DB
engine = db.create_engine(db_azure)
conn = engine.raw_connection()

In [ ]:
# Create new table in PostgreSQL
commands = (f'''CREATE TABLE IF NOT EXISTS location(
);''')

# Initialize connection to PostgreSQL
cur = conn.cursor()

# Create cursor to execute SQL commands
#for command in commands:
cur.execute(commands)

# Commit changes
conn.commit()

In [ ]:
# Copy data to table
geo_df.to_sql(name= 'location', con = engine, if_exists= 'replace')

In [ ]:
# Close communication with server
cur.close()
conn.close()

# End - Random String Checks

In [ ]:
print(geo_df[geo_df['city'] == 'Abadia Dos Dourados']['city'].count())

In [ ]:
for ele in sorted(city):
    print(ele)